In [14]:
import bs4 as bs
import urllib.request
import pandas as pd
import numpy as np

In [15]:
# Make race result
def MakeRaceResult (url_r, season_name, race_name):
  # Get Source
  source = urllib.request.urlopen(url_r)
  # HTML to table
  soup = bs.BeautifulSoup(source, "lxml")
  table = soup.find_all("table")[0]
  df = pd.read_html(str(table), flavor="bs4", header=[0])[0]
  # Remove strange row
  df_n = df.drop(df[(df.index % 2 == 1)].index)
  # Remove unused column
  df_l = df_n.drop(['MoreMore', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'], axis=1)
  # Reset index
  df_l = df_l.reset_index(drop=True)

  # Get Source again
  source = urllib.request.urlopen(url_r)
  # Read HTML again for Driver Name
  soup = bs.BeautifulSoup(source, "lxml")

  # Make table for driver's first name
  tb_fname = soup.find_all("div", {"class":"driver__fname"})
  df_fname = pd.DataFrame(tb_fname)
  df_fname.rename(columns={0: 'DriverFirstName'}, inplace=True)
  # Remove duplicated row
  df_fname = df_fname.drop(df_fname[(df_fname.index % 2 == 1)].index)
  df_fname = df_fname.reset_index(drop=True)

  # Make table for driver's last name
  tb_lname = soup.find_all("span", {"class":"full"})
  df_lname = pd.DataFrame(tb_lname)
  df_lname.rename(columns={0: 'DriverLastName'}, inplace=True)
  # Remove duplicated row
  df_lname = df_lname.drop(df_lname[(df_lname.index % 2 == 1)].index)
  df_lname = df_lname.reset_index(drop=True)

  # Make table for driver's number
  tb_number = soup.find_all("div", {"class":"driver__number"})
  df_number = pd.DataFrame(tb_number)
  df_number.rename(columns={0: 'DriverNumber'}, inplace=True)
  # Remove duplicated row
  df_number = df_number.drop(df_number[(df_number.index % 2 == 1)].index)
  df_number = df_number.reset_index(drop=True)

  # Concat race result and name and number
  df_result = pd.concat([df_l, df_number, df_fname, df_lname], axis=1)

  # Changing column name
  df_result.rename(columns={'PosPos':'Pos', 'TeamTeam':'Team', 'StartedStarted':'Started', 'BestBest':'Best', 'TimeTime':'Time'}, inplace=True)
  # Removing Driver column
  df_result = df_result.drop(['DriverDriver'], axis=1)

  # Add season name
  df_result['SeasonName'] = season_name
  df_result['RaceName'] = race_name

  # Changing column order
  df_result = df_result[['SeasonName','RaceName','Pos','DriverNumber','DriverFirstName','DriverLastName','Team','Started','Best', 'Time','PtsPoints']]

  return df_result

In [16]:
# Main process
# Get Season & Race information (Pre-writtened)
race_list = pd.read_csv('data/formula_e_race_infomation.csv')

In [17]:
# Make season & race url -> Call main function
race_result = pd.DataFrame()
for item in race_list.itertuples():
  race_url = "https://www.fiaformulae.com/en/results/race-results/?championship=" + str(getattr(item, "SeasonCode")) + \
  "&race=" + str(getattr(item, "RaceCode"))
  s_name = getattr(item, "SeasonName")
  r_name = getattr(item, "RaceName")
  # main process
  r_result = MakeRaceResult(race_url, s_name, r_name)
  race_result = race_result.append(r_result)

In [18]:
race_result

,SeasonName,RaceName,Pos,DriverNumber,DriverFirstName,DriverLastName,Team,Started,Best,Time,PtsPoints
0,Season 1 2014/15,Beijing E-Prix,1,#11,Lucas,Di Grassi,Audi Sport ABT Formula E Team,2,1:46.718,52:23.413,25
1,Season 1 2014/15,Beijing E-Prix,2,#27,Franck,Montagny,Andretti Autosport Formula E Team,8,1:46.640,+ 2.867,18
2,Season 1 2014/15,Beijing E-Prix,3,#2,Sam,Bird,Virgin Racing Formula E Team,11,1:46.563,+ 6.559,15
3,Season 1 2014/15,Beijing E-Prix,4,#28,Charles,Pic,Andretti Autosport Formula E Team,7,1:46.730,+ 19.301,12
4,Season 1 2014/15,Beijing E-Prix,5,#5,Karun,Chandhok,Mahindra Racing Formula E Team,4,1:45.892,+ 23.952,10
...,...,...,...,...,...,...,...,...,...,...,...
19,Season 7 2020/21,Monaco E-Prix - R7,0,#17,Nyck,De Vries,Mercedes-EQ Formula E Team,24,1:36.102,–DNF,0
20,Season 7 2020/21,Monaco E-Prix - R7,0,#99,Pascal,Wehrlein,TAG Heuer Porsche Formula E Team,8,1:35.717,–DNF,0
21,Season 7 2020/21,Monaco E-Prix - R7,0,#5,Stoffel,Vandoorne,Mercedes-EQ Formula E Team,15,1:34.428,–DNF,0
22,Season 7 2020/21,Monaco E-Prix - R7,0,#33,René,Rast,Audi Sport ABT Schaeffler,10,1:34.586,–DNF,0


In [20]:
# Result file
race_result.to_csv('out\Formual_E_Raceresults.csv', index=False)